In [1]:
#Import libraries 
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as k
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle

Using TensorFlow backend.


In [2]:
#Load the dataset 
df = pd.read_csv('reviews_dataset.tsv.zip',header=0, delimiter="\t", quoting=3)

# Keeping only the neccessary columns
df = df[['review','sentiment']]

In [3]:
#shape of the data 
df.shape

(25000, 2)

In [4]:
df.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [5]:
df['review'] = df['review'].apply(lambda x: x.lower())
df['review'] = df['review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))



In [6]:
max_features = 1000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['review'].values)
X = tokenizer.texts_to_sequences(df['review'].values)
X = pad_sequences(X)

In [7]:
X.shape

(25000, 1473)

In [8]:
embed_dim = 50
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(LSTM(10))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1473, 50)          50000     
_________________________________________________________________
lstm (LSTM)                  (None, 10)                2440      
_________________________________________________________________
dense (Dense)                (None, 2)                 22        
Total params: 52,462
Trainable params: 52,462
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
y = pd.get_dummies(df['sentiment']).values
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 99)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(18750, 1473) (18750, 2)
(6250, 1473) (6250, 2)


In [10]:
model.fit(X_train, y_train, epochs = 5, verbose = 1)

Train on 18750 samples
Epoch 1/5
18750/18750 [==============================] - 263s 14ms/sample - loss: 0.4837 - accuracy: 0.7610
Epoch 2/5
18750/18750 [==============================] - 278s 15ms/sample - loss: 0.3485 - accuracy: 0.8545
Epoch 3/5
18750/18750 [==============================] - 273s 15ms/sample - loss: 0.3279 - accuracy: 0.8623
Epoch 4/5
18750/18750 [==============================] - 274s 15ms/sample - loss: 0.3077 - accuracy: 0.8709
Epoch 5/5
18750/18750 [==============================] - 270s 14ms/sample - loss: 0.2922 - accuracy: 0.8765


In [16]:
# save tokenizer for preprocessing
with open('tokenizer.pickle', 'wb') as tk:
    pickle.dump(tokenizer, tk, protocol=pickle.HIGHEST_PROTOCOL)
# serialize model to JSON for Flask App
model_json = model.to_json()
with open("model.json", "w") as js:
    js.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [15]:
test = ['Movie was good but awful']
#vectorizing the tweet by the pre-fitted tokenizer instance
test = tokenizer.texts_to_sequences(test)
#padding the tweet to have exactly the same shape as `embedding_2` input
test = pad_sequences(test, maxlen=X.shape[1], dtype='int32', value=0)
print(test.shape)
sentiment = model.predict(test)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

(1, 1473)
negative
